# QENS data reduction - DMSC Summer School

In [ ]:
import numpy as np
import scipp as sc
import plopp as pp
import os
import utils

## Process run with sample

In [ ]:
top_folder = 'McStas_QENS_NeXus_with_issue'
run_folder = 'QENS_elastic_NeXus_1_pulse'
fname = os.path.join(top_folder, run_folder, 'mccode.h5')

In [ ]:
events, params, dg = utils.load_nexus(fname)

In [ ]:
params

In [ ]:
dg

In [ ]:
sample = utils.load_nexus(fname)
sample

In [ ]:
sample.hist(tof=200, y=200).plot(norm='log', vmin=1.0e-2)

In [ ]:
sample.hist(y=200).plot(norm='log')

In [ ]:
from scippneutron.conversion.graph.beamline import beamline
from scippneutron.conversion.graph.tof import elastic

In [ ]:
graph = beamline(scatter=True)
sc.show_graph(graph, simplified=True)

In [ ]:
def scat(sample_position, analyzer_position):
    return analyzer_position - sample_position

def l3(analyzer_position, position):
    return position - analyzer_position

def ltot(L1, L2, L3):
    return L1 + L2 + L3
graph['scattered_beam'] = scat
graph['Ltotal'] = ltot
graph['L3'] = l3

In [ ]:
sc.show_graph(graph, simplified=True)

In [ ]:
graph = {**beamline(scatter=True), **elastic("tof")}
sc.show_graph(graph, simplified=True)

In [ ]:
sample.transform_coords("wavelength", graph=graph).hist(wavelength=200).plot()

In [ ]:
sample_q = sample.transform_coords("Q", graph=graph)
sample_q

In [ ]:
sample_h = sample_q.hist(Q=200)
sample_h

### Add variances to the data

In [ ]:
utils.add_variances(sample_h)
sample_h.data

In [ ]:
sample_h.plot(norm='log')

## Process flat-field run

In [ ]:
run_folder = 'SANS_without_sample_NeXus_1_pulse'
fname = os.path.join(top_folder, run_folder, 'mccode.h5')

flat = utils.load_nexus(fname, 5.0e4)

In [ ]:
flat.hist(y=200).plot()

In [ ]:
pp.plot({'sample': sample.hist(y=200),
         'flat-field': flat.hist(y=200)},
       norm='log')

In [ ]:
flat_q = flat.transform_coords("Q", graph=graph)

In [ ]:
flat_h = flat_q.hist(Q=200)
utils.add_variances(flat_h)
flat_h.plot()

## Normalize

In [ ]:
normed = sample_h / flat_h

In [ ]:
qbins = sc.linspace('Q', 5.0e-3, 0.2, 201, unit='1/angstrom')
qbins

In [ ]:
sample_h = sample_q.hist(Q=qbins)
flat_h = flat_q.hist(Q=qbins)

utils.add_variances(sample_h, flat_h)

normed = sample_h / flat_h
normed

In [ ]:
normed.plot(norm='log')

## Save result to disk

In [ ]:
q = sc.midpoints(normed.coords['Q']).values
data = np.array([q, normed.values, np.sqrt(normed.variances)]).T

np.savetxt('sans_iofq.dat', data, header="Q [1/A], I, stddev", delimiter=' ')